## Q1: The stock market

(This is about numba)

A Markov Chain is defined as a sequence of random variables where a parameter depends *only* on the preceding value. This is a crucial tool in statistics, widely used in science and beyond (economics for instance).

For instance, the stock market has phases of growing prices (bull), dreasing prices (bear) and recession. This would be a Markov Chain model:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Finance_Markov_chain_example_state_space.svg/400px-Finance_Markov_chain_example_state_space.svg.png)

where the numbers on the arrows indicate the probability that the next day will be in a given state.

Your task is to simulate the stock market according to this rule. Start from a random state and simulate many many  iterations. If your code is right, the fraction of days in each state should converge. 

Implement a pure-python version and a numba version, and compare speeds. 


In [143]:
import numpy as np
import time
from numba import njit

In [145]:
# Define the probability of the transition matrix
probability = np.array([
    [0.7, 0.2, 0.1],  # Bull -> Bull, Bull -> Bear, Bull -> Recession
    [0.3, 0.6, 0.1],  # Bear -> Bull, Bear -> Bear, Bear -> Recession
    [0.3, 0.2, 0.5]   # Recession -> Bull, Recession -> Bear, Recession -> Recession
])

# States:
states = np.array([0, 1, 2]) # ["Bull", "Bear", "Recession"]

In [147]:
# Pure Python version
def markov_python(states, probability, steps):
    current_state = np.random.choice(states)  # Start in a random state
    state_counts = np.zeros(len(states))  # Count occurrences of each state

    for _ in range(steps):
        state_counts[current_state] += 1
        current_state = np.random.choice(states, p=probability[current_state])

    return state_counts / steps  # Return fractions of time spent in each state

In [159]:
# Numba version
@njit
def markov_numba(states, probability, steps):
    current_state = np.random.choice(states)  # Start in a random state
    state_counts = np.zeros(len(states))  # Count occurrences of each state

    # Numba random number generation
    for _ in range(steps):
        state_counts[current_state] += 1
        rand_val = np.random.random()
        cumulative_prob = 0.0
        for i in range(len(states)):
            cumulative_prob += probability[current_state, i]
            if rand_val < cumulative_prob:
                current_state = states[i]
                break

    return state_counts / steps  # Return fractions of time spent in each state

In [166]:
_ = markov_numba(states, probability, 10) # Warm up Numba
steps = 100000  # Number of iterations

# Compare speeds
# Pure Python
start_time = time.time()
fractions_python = markov_python(states, probability, steps)
python_time = time.time() - start_time

# Numba
start_time = time.time()
fractions_numba = markov_numba(states, probability, steps)
numba_time = time.time() - start_time

In [168]:
# Print results
print("\nPure Python Results:")
for i, state in enumerate(["Bull", "Bear", "Recession"]):
    print(f"{state}: {fractions_python[i]:.4f}")
print(f"Time taken: {python_time:.4f} seconds\n")

print("Numba Results:")
for i, state in enumerate(["Bull", "Bear", "Recession"]):
    print(f"{state}: {fractions_numba[i]:.4f}")
print(f"Time taken: {numba_time:.4f} seconds\n")

print(f"Numba is {python_time/numba_time:.2f}x faster than Pure Python.")


Pure Python Results:
Bull: 0.4993
Bear: 0.3352
Recession: 0.1655
Time taken: 7.1401 seconds

Numba Results:
Bull: 0.4981
Bear: 0.3338
Recession: 0.1681
Time taken: 0.0040 seconds

Numba is 1774.69x faster than Pure Python.
